In [1]:
import os
import json
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn import svm

from pmlb import fetch_data

## Read Data
You can change the code below to read your own data.

In [11]:
data_name = 'obesity_level'
file_path = "input/ObesityDataSet_raw_and_data_sinthetic/ObesityDataSet_raw_and_data_sinthetic.csv"

# Returns a pandas DataFrame
obj_df = pd.read_csv(filepath_or_buffer=file_path, index_col=None, header=0)

In [12]:
obj_df.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


In [13]:
obj_df['NObeyesdad'].value_counts()

Obesity_Type_I         351
Obesity_Type_III       324
Obesity_Type_II        297
Overweight_Level_II    290
Overweight_Level_I     290
Normal_Weight          287
Insufficient_Weight    272
Name: NObeyesdad, dtype: int64

In [15]:
cleanup_nums = {"Gender": {"Male": 0, "Female": 1},
                "family_history_with_overweight": {"no": 0, "yes": 1},
                "FAVC": {"no": 0, "yes": 1},
                "CAEC": {"no": 0, "Sometimes": 1, "Frequently": 2, "Always": 3},
                "SMOKE": {"no": 0, "yes": 1},
                "SCC": {"no": 0, "yes": 1},
                "CALC": {"no": 0, "Sometimes": 1, "Frequently": 2, "Always": 3},
                "MTRANS": {"Walking": 0, "Bike": 1, "Motorbike": 2, "Public_Transportation": 3, "Automobile": 4},
                "NObeyesdad": {
                    "Insufficient_Weight":0,
                    "Normal_Weight": 1,
                    "Overweight_Level_I": 2,
                    "Overweight_Level_II": 3,
                    "Obesity_Type_I": 4,
                    "Obesity_Type_II": 5,
                    "Obesity_Type_III": 6,
                }
            }

In [16]:
df = obj_df.replace(cleanup_nums)
df.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,1,21.0,1.62,64.0,1,0,2.0,3.0,1,0,2.0,0,0.0,1.0,0,3,1
1,1,21.0,1.52,56.0,1,0,3.0,3.0,1,1,3.0,1,3.0,0.0,1,3,1
2,0,23.0,1.80,77.0,1,0,2.0,3.0,1,0,2.0,0,2.0,1.0,2,3,1
3,0,27.0,1.80,87.0,0,0,3.0,3.0,1,0,2.0,0,2.0,0.0,2,0,2
4,0,22.0,1.78,89.8,0,0,2.0,1.0,1,0,2.0,0,0.0,0.0,1,3,3


In [17]:
df.describe()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
count,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000
mean,0.494079,24.312600,1.701677,86.586058,0.817622,0.883941,2.419043,2.685628,1.140692,0.020843,2.008011,0.045476,1.010298,0.657866,0.731407,3.125059,3.112269
std,0.500083,6.345968,0.093305,26.191172,0.386247,0.320371,0.533927,0.778039,0.468543,0.142893,0.612953,0.208395,0.850592,0.608927,0.515498,0.676969,1.985062
min,0.000000,14.000000,1.450000,39.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,19.947192,1.630000,65.473343,1.000000,1.000000,2.000000,2.658738,1.000000,0.000000,1.584812,0.000000,0.124505,0.000000,0.000000,3.000000,1.000000
50%,0.000000,22.777890,1.700499,83.000000,1.000000,1.000000,2.385502,3.000000,1.000000,0.000000,2.000000,0.000000,1.000000,0.625350,1.000000,3.000000,3.000000
75%,1.000000,26.000000,1.768464,107.430682,1.000000,1.000000,3.000000,3.000000,1.000000,0.000000,2.477420,0.000000,1.666678,1.000000,1.000000,3.000000,5.000000
max,1.000000,61.000000,1.980000,173.000000,1.000000,1.000000,3.000000,4.000000,3.000000,1.000000,3.000000,1.000000,3.000000,2.000000,3.000000,4.000000,6.000000


In [18]:
df.shape

(2111, 17)

In [19]:
df['NObeyesdad'].value_counts()

4    351
6    324
5    297
3    290
2    290
1    287
0    272
Name: NObeyesdad, dtype: int64

In [34]:
np.unique(df['NObeyesdad'], return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6]), array([272, 287, 290, 290, 351, 297, 324]))

## Train a classification model
In this step, you can either train a model in the cell below, or skip the training part and read the model prediction of the training data from your own model .

In [20]:
to_keep = df.columns[:-1]
to_keep

Index(['Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight',
       'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE',
       'CALC', 'MTRANS'],
      dtype='object')

In [21]:
'''prepare data'''
target_col = 'NObeyesdad'

X = df[to_keep].values
y = df[target_col].values
y = y.reshape(len(y))

train, test, train_labels, test_labels = train_test_split(X, y, test_size = 0.2)

train_df = pd.DataFrame(train, columns=to_keep)
test_df = pd.DataFrame(test, columns=to_keep)


In [29]:
'''train the model'''
clf = MLPClassifier(random_state=1, hidden_layer_sizes= (50), max_iter=3000)
clf.fit(train, train_labels)

'''report accuracy'''
print(clf.score(test,test_labels ))

0.8297872340425532


In [30]:
y_pred = clf.predict(train)

In [31]:
np.unique(y_pred, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6]), array([238, 219, 232, 185, 302, 247, 265]))

## Output Training Data

In [32]:
'''
cols: the column names of the input data.
data: the training input.
target_names: the names for the target classes (grount truth).
real_min: minimal values for columns of the input data.
real_max: maximal values for columns of the input data.
y_pred: the prediction of the input data from the model you want to explain.
y_gt: the ground truth of the input data.
'''

def output_data(cols, data, target_names, real_min, real_max, y_pred, y_gt):
    filename = "./output/"+data_name+"/test.json"
    directory = os.path.dirname(filename)
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    to_output = {}
    to_output['columns'] = cols
    to_output['data'] = data
    to_output['target_names'] = target_names
    to_output['real_min'] = real_min
    to_output['real_max'] = real_max
    to_output['y_pred'] = y_pred
    to_output['y_gt'] = y_gt
    to_output['n_cls'] = 7
    with open(filename, 'w') as output:
        output.write(json.dumps(to_output))

In [33]:
''' name the target classes '''
target_names = ["Insufficient_Weight",
                    "Normal_Weight",
                    "Overweight_Level_I",
                    "Overweight_Level_II",
                    "Obesity_Type_I",
                    "Obesity_Type_II",
                    "Obesity_Type_III",]

''' obtain the min and max values '''
min_val = np.min(X, axis=0)
max_val = np.max(X, axis=0)

''' output data '''
output_data(cols=to_keep.values.tolist(), 
            data=train.tolist(), 
            target_names=target_names, 
            real_min=min_val.tolist(), 
            real_max=max_val.tolist(),
            y_pred=y_pred.tolist(), 
            y_gt=train_labels.tolist())